In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, when, col, to_date, date_format
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType, MapType

spark = SparkSession.builder \
    .appName("KafkaStreamingExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.4") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

# Kafka connection details
bootstrap_servers = "pkc-56d1g.eastus.azure.confluent.cloud:9092"
kafka_topic = "salma_topic"
kafka_username = "JUKQQM4ZM632RECA"
kafka_password = "UUkrPuSttgOC0U9lY3ZansNsKfN9fbxZPFwrGxudDrfv+knTD4rCwK+KdIzVPX0D"

# Define schema for the incoming JSON data
schema = StructType() \
    .add("eventType", StringType()) \
    .add("customerId", StringType()) \
    .add("productId", StringType()) \
    .add("timestamp", StringType()) \
    .add("metadata", MapType(StringType(), StringType())) \
    .add("quantity", IntegerType()) \
    .add("totalAmount", DoubleType()) \
    .add("paymentMethod", StringType()) \
    .add("recommendedProductId", StringType()) \
    .add("algorithm", StringType())

# Read data from Kafka topic as a streaming DataFrame
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{kafka_username}" password="{kafka_password}";') \
    .load()

json_df = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")) \
    .select("data.*")

# Handle schema based on event type and derive date and hour columns
handled_df = json_df \
    .withColumn("timestamp", col("timestamp").cast("timestamp")) \
    .withColumn("quantity", when(col("eventType") == "purchase", col("quantity"))
                .when(col("eventType") == "addToCart", col("quantity"))
                .otherwise(0)) \
    .withColumn("totalAmount", when(col("eventType") == "purchase", col("totalAmount")).otherwise(0.0)) \
    .withColumn("paymentMethod", when(col("eventType") == "purchase", col("paymentMethod")).otherwise("Unknown")) \
    .withColumn("recommendedProductId", when(col("eventType") == "recommendationClick", col("recommendedProductId"))
                .otherwise("None")) \
    .withColumn("algorithm", when(col("eventType") == "recommendationClick", col("algorithm")).otherwise("None")) \
    .withColumn("date", to_date(col("timestamp"))) \
    .withColumn("hour", date_format(col("timestamp"), "HH"))

# Write data to Parquet format partitioned by date and hour
query = handled_df \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/user/itversity/prj") \
    .option("checkpointLocation", "/user/itversity/checkpoint") \
    .partitionBy("date", "hour") \
    .start()

query.awaitTermination()


/home/itversity/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/itversity/.local/lib/python3.6/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=54>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/itversity/.local/lib/python3.6/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/itversity/.local/lib/python3.6/site-packages/py4j/clientserver.py", line 540, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending com

Py4JError: An error occurred while calling o111.awaitTermination